# Agregar un encabezado de clasificación

Durante esta sección se modificará el modelo LLM preentrenado para prepararlo para el ajuste fino de la clasificación. Para ello se reemplaza la  capa  de  salida  original,  que  asigna  la  representación  oculta  a  un  vocabulario  de  50 257,  por  una  capa  de  salida  más  pequeña  que  asigna  dos  clases:  0  («no  spam»)  y  1  («spam»).

![Texto alternativo](./imgs/6.8.png)

Como se ve en la figura  se va a utilizar  el  mismo  modelo  que  en  las  secciones  anteriores  excepto  que  se reemplaza la  capa  de  salida.

Técnicamente,  se podría  usar  un  solo  nodo  de  salida,  dado  que  se  trata  de  una  tarea  de  
clasificación  binaria. Sin  embargo,  esto  requeriría  modificar  la  función  de  pérdida.
Se opta por  un  enfoque  más  general  donde  el  número  de  nodos  de  salida  coincide  con  el  número  de  clases.  Por  ejemplo,  para  un  problema  de  tres  clases,  como  clasificar  artículos  de  noticias  como  "Tecnología",  "Deportes"  o  "Política",  se usarían tres  nodos  de  salida,  y  así  sucesivamete.

Antes de modificar el modelo, se va a volver a cargar y a mostrar:

In [1]:
import sys
import os

# Obtiene la ruta de la carpeta principal del proyecto (subiendo un nivel desde seccion05)
ruta_proyecto_principal = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Añade esta ruta a la lista de lugares donde Python busca módulos
if ruta_proyecto_principal not in sys.path:
    sys.path.append(ruta_proyecto_principal)

In [2]:
from seccion05_PreentrenamientoDatosNoEtiquetados.gpt_download import download_and_load_gpt2
from seccion05_PreentrenamientoDatosNoEtiquetados.loadWeightsGPTModels import load_weights_into_gpt
from seccion04_ImplementacionGPTGeneracionTexto.gptModel import GPTModel
#Reeutilizar configuraciones
CHOOSE_MODEL = "gpt2-small (124M)"
INPUT_PROMPT = "Every effort moves"
BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}
BASE_CONFIG.update(model_configs[CHOOSE_MODEL])
model_size = CHOOSE_MODEL.split(" ")[-1].lstrip("(").rstrip(")")
settings, params = download_and_load_gpt2(model_size=model_size, models_dir="gpt2")
model = GPTModel(BASE_CONFIG)

print(model)

File already exists and is up-to-date: gpt2\124M\checkpoint
File already exists and is up-to-date: gpt2\124M\encoder.json
File already exists and is up-to-date: gpt2\124M\hparams.json
File already exists and is up-to-date: gpt2\124M\model.ckpt.data-00000-of-00001
File already exists and is up-to-date: gpt2\124M\model.ckpt.index
File already exists and is up-to-date: gpt2\124M\model.ckpt.meta
File already exists and is up-to-date: gpt2\124M\vocab.bpe
GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.0, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=True)
        (W_key): Linear(in_features=768, out_features=768, bias=True)
        (W_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)

El  GPTModel  consta  de  capas  de  incrustación,  seguidas  de  12  bloques  de  transformadores  idénticos,  seguidos  de  una  capa LayerNorm  final  y  la  capa  de  salida,  out_head.

### AJUSTE  DE  CAPAS  SELECCIONADAS  FRENTE  A  TODAS  LAS  CAPAS
Dado  que se parte de  un  modelo  preentrenado,  no  es  necesario  ajustar  todas  las  capas  del  modelo.  Esto  se  debe  a  que,  en  los  modelos  de  lenguaje  basados  en  redes  neuronales,  las  capas  inferiores  generalmente  capturan  las  estructuras  y  la  semántica  básicas  del  lenguaje,  aplicables  a  una  amplia  gama  de  tareas  y  conjuntos  de  datos.  Por  lo  tanto,  ajustar  solo  las  últimas  capas  (capas  cercanas  a  la  salida),  que  son  más  específicas  para  
patrones  lingüísticos  con  matices  y  características  específicas  de  la  tarea,  suele  ser  suficiente  para  adaptar  el  modelo  a  nuevas  tareas.  Un  efecto  secundario  positivo  es  que  resulta  computacionalmente  más  eficiente  ajustar  solo  un  número  reducido  de  capas.

Para  preparar  el  modelo  para  el  ajuste  de  la  clasificación,  primero  se congela  el  modelo,  lo  que  significa  que  se hace  que  todas  las  capas  no  se  puedan  entrenar:

In [3]:
for param in model.parameters():
    param.requires_grad = False

Reemplazar la capa de salida (model.out_head), que  originalmente  asigna  las  entradas  de  la  capa  a  50,257  dimensiones  (el  tamaño  del  vocabulario):

In [6]:
import torch
torch.manual_seed(123)
num_classes = 2
model.out_head = torch.nn.Linear(
    in_features=BASE_CONFIG["emb_dim"],
    out_features=num_classes
)

Esta  nueva  capa  de  salida  model.out_head  tiene  su  atributo  requires_grad  establecido  en  Verdadero  de  manera  predeterminada,  lo  que  significa  que  es  la  única  capa  del  modelo  que  se  actualizará  durante  el  entrenamiento.

Técnicamente,  entrenar  la  capa  de  salida  que  se acaba  de  añadir  es  suficiente.  Sin  embargo,  como se han descubierto en experimentos,  ajustar  capas  adicionales  puede  mejorar  notablemente  el  rendimiento  predictivo  del  modelo  ajustado. 

Además,  se configura el  último  bloque  transformador  y  el  módulo  final  LayerNorm ,que  conecta  este  bloque  a  la  capa  de  salida,  para  que  sea  entrenable.


![Texto alternativo](./imgs/6.9.png)

Para  hacer  que  el  último  bloque  LayerNorm  y  el  último  transformador  sean  entrenables,  se establecen  sus  respectivos  requires_grad  en  True.

In [7]:
for param in model.trf_blocks[-1].parameters():
    param.requires_grad = True

for param in model.final_norm.parameters():
    param.requires_grad = True

Aunque se añade una  nueva  capa  de  salida  y  se marcan  ciertas  capas  como  entrenables  o  no,  se puede usar  este  modelo  de  forma  similar  a  las secciones anteriores. 

In [8]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
inputs = tokenizer.encode("Do you have time")
inputs = torch.tensor(inputs).unsqueeze(0)
print("Inputs:", inputs)
print("Inputs dimensions:", inputs.shape) # shape: (batch_size, num_tokens)

Inputs: tensor([[5211,  345,  423,  640]])
Inputs dimensions: torch.Size([1, 4])


Como  lo  muestra  la  salida  de  impresión,  el  código  anterior  codifica  las  entradas  en  un  tensor  que  consta  de  4  tokens  de  entrada.

Luego se pueden pasar los ID de token codificados al modelo como se ha hecho:

In [ ]:
with torch.no_grad():
    outputs = model(inputs)
print("Outputs:\n", outputs)
print("Outputs dimensions:", outputs.shape) #shape: (batch_size, num_tokens, num_classes)   

Outputs:
 tensor([[[-0.5956, -0.0469],
         [-0.4593, -0.5447],
         [-0.1016,  0.5089],
         [-0.0080, -0.4943]]])
Outputs dimensions: torch.Size([1, 4, 2])


En secciones anteriores,  una  entrada  similar  habría  generado  un  tensor  de  salida  de  [1,  4,  50257],  donde  50257  representa  el  tamaño  del  vocabulario.  Al  igual  que  en  secciones anteiores,  el  número  de  filas  de  salida  corresponde  al  número  de  tokens  de  entrada  (en  este  caso,  4).  Sin  embargo,  la  dimensión  de  incrustación  de  cada  salida  (el  número  de  columnas)  se  ha  reducido  a  2  en  lugar  de  50257,  ya  que  se ha reemplazado  la  capa  de  salida  del  modelo.

Solo interesa ajustar  este  modelo  para  que  devuelva  una  etiqueta  de  clase  que  indique  si  una  entrada  del  modelo  es  spam  o  no.  Para  lograrlo,  no se necesita ajustar las  cuatro  filas  de  salida,  sino  que se puede centrar  en  un  solo  token  de  salida.  En  particular,  hay que centrarse  en  la  última  fila  correspondiente  al  último  token  de  salida.

![Texto alternativo](./imgs/6.10.png)
